# ETL Proyecto Individual ML Ops STEAM

In [1]:
# Importo las librerias que considero necesarias para hacer la extración de los Data Set
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Ruta de ubicación para los datasets:
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_user_reviews.json'
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_users_items.json'
# 'E:\\AAADATOS\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\output_steam_games.json'

## Data Set australian_user_reviews.json

In [13]:
# Se inicializa una lista vacia donde alamcenare los datso del archivo .JSON
filas = []
# Se abre el archivo .JSON con el modulo 'with'
with open('E:\\AAADATOS\\Henry\\AA_Data_Science\\MATERIAL_PI\\DATA_PI MLOps - STEAM\\Data_sets\\australian_user_reviews.json', 'r', encoding='utf-8') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

In [14]:
# Se inicializa el primer DataFrame a partir de la lista donde se extrajo el archivo .JSON
df_reviews = pd.DataFrame(filas)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se evidencia que la columna "reviews" contiene listas como dato así que se tiene que proceder a expandirla

In [15]:
df_reviews = df_reviews.explode('reviews').reset_index() # Se redefine el DatFrame con una descomposición del campo para desanidarla y se reinicia el conteo del índice
df_reviews = df_reviews.drop(columns='index') # Se elimina el campo 'index'
df_reviews = pd.concat([df_reviews, pd.json_normalize(df_reviews['reviews'])], axis=1) # Se normalizan los datos JSON de la columna 'reviews' aplanando los datos, convirtiendolos en un formato tabular y concatenando de manera horizontal el DatFrame original con el del contendio de 'reviews' 
df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


### Se empieza a analizar y transformar los datos del DataFrame

In [18]:
# Información general del DataFrame
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


NULOS

In [20]:
# Conteo de Nulos
nulos = df_reviews.isna().sum()
nulos

user_id         0
user_url        0
reviews        28
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

In [22]:
# Visualizar los valores Nulos
df_nulos = df_reviews[df_reviews.isnull().any(axis=1)] # Se crea un nuevo DatFrame donde se filtra si hay algun valor nulo en la fila a partir del DataFrame orginal
df_nulos

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
137,gdxsd,http://steamcommunity.com/id/gdxsd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,76561198094224872,http://steamcommunity.com/profiles/76561198094...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2559,76561198021575394,http://steamcommunity.com/profiles/76561198021...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,cmuir37,http://steamcommunity.com/id/cmuir37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13767,Jaysteeny,http://steamcommunity.com/id/Jaysteeny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15493,ML8989,http://steamcommunity.com/id/ML8989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19184,76561198079215291,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25056,76561198061996985,http://steamcommunity.com/profiles/76561198061...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26257,76561198108286351,http://steamcommunity.com/profiles/76561198108...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En base al DataFrame anterior se puede afirmar que todos los Nulos corresponden estas filas donde no esta la información relevante para mis objetivos con las consignas que se me asignaron por lo cual se procedera a ser eliminados

In [24]:
# Elimino las filas con valores nulos del DataFrame con el método 'dropna()'
df_reviews.dropna(inplace=True)
# Conteo de Nulos nuevamente para verificar
nuevos_nulos = df_reviews.isna().sum()
nuevos_nulos

user_id        0
user_url       0
reviews        0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

De esta manera se han eliminado los valores Nulos del DataFrame hasta este punto de la transformación

#### Se procede a evaluar el contenido de los campos y determinar su relevancia dentro de los obejtivos

In [29]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   reviews      59305 non-null  object
 3   funny        59305 non-null  object
 4   posted       59305 non-null  object
 5   last_edited  59305 non-null  object
 6   item_id      59305 non-null  object
 7   helpful      59305 non-null  object
 8   recommend    59305 non-null  object
 9   review       59305 non-null  object
dtypes: object(10)
memory usage: 5.0+ MB


In [28]:
# Se crea una variable que contiene los valores unicos del campo 'funny'
valores_funny = df_reviews['funny'].unique() 
valores_funny

array(['', '3 people found this review funny',
       '1 person found this review funny',
       '2 people found this review funny',
       '5 people found this review funny',
       '17 people found this review funny',
       '524 people found this review funny',
       '4 people found this review funny',
       '12 people found this review funny',
       '6 people found this review funny',
       '7 people found this review funny',
       '103 people found this review funny',
       '16 people found this review funny',
       '34 people found this review funny',
       '9 people found this review funny',
       '45 people found this review funny',
       '11 people found this review funny',
       '97 people found this review funny',
       '207 people found this review funny',
       '718 people found this review funny',
       '23 people found this review funny',
       '100 people found this review funny',
       '8 people found this review funny',
       '19 people found this rev

In [30]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_funny = df_reviews[df_reviews['funny'] != ''].count()['funny']
cant_funny

8151

In [33]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de funny completos dentro del conjunto del DataFrame
total_filas_df = 59305
print(f'La cantidad de valores que no son strings vacios en el campo "funny" representan un {round((cant_funny/total_filas_df)*100, 2)}%')

La cantidad de valores en el campo "funny" representan un 13.74%


Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "funny" apenas representan un 13.74% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [35]:
# Se crea una variable que contiene los valores unicos del campo 'last_edited'
valores_last_edited = df_reviews['last_edited'].unique() 
valores_last_edited

array(['', 'Last edited December 5, 2015.',
       'Last edited November 25, 2013.', ...,
       'Last edited April 17, 2014.', 'Last edited May 10, 2014.',
       'Last edited August 15, 2014.'], dtype=object)

In [36]:
# Se crea otra variable para contar cuantos valores hay que no sean strings vacios ("")
cant_last_edited = df_reviews[df_reviews['last_edited'] != ''].count()['last_edited']
cant_last_edited

6140

In [37]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "last_edited" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son strings vacios en el campo "last_edited" representan un {round((cant_last_edited/total_filas_df)*100, 2)}%')

La cantidad de valores que no son strings vacios en el campo "last_edited" representan un 10.35%


Después del proceso anterior se puede determinar que los valores que se podrían considerar útiles del campo "last_edited" apenas representan un 10.35% dentro de todo el DataFrame por lo cual este campo en su conjunto se considerará presindible

In [39]:
# Se crea una variable que contiene los valores unicos del campo 'helpful'
valores_helpful = df_reviews['helpful'].unique() 
valores_helpful

array(['No ratings yet',
       '15 of 20 people (75%) found this review helpful',
       '0 of 1 people (0%) found this review helpful', ...,
       '21 of 34 people (62%) found this review helpful',
       '64 of 73 people (88%) found this review helpful',
       '24 of 34 people (71%) found this review helpful'], dtype=object)

In [40]:
# Se crea otra variable para contar cuantos valores hay que no sean "No ratings yet"
cant_helpful = df_reviews[df_reviews['helpful'] != 'No ratings yet'].count()['helpful']
cant_helpful

29137

In [42]:
# Se hace una operación matematica para determinar que porcentaje representa los valores de "helpful" completos dentro del conjunto del DataFrame
print(f'La cantidad de valores que no son "No ratings yet" vacios en el campo "helpful" representan un {round((cant_helpful/total_filas_df)*100, 2)}%')

La cantidad de valores que no son "No ratings yet" vacios en el campo "helpful" representan un 49.13%


Ahora que se posee mas información sobre el campo "helpful" se puede determinar que es presindible no solo por que su información relevante represente un poco menos del 50%, sino también por que esta información 'útil' seria dificil de cuantificar y llevar a escala además en los requerimientos se indica que el campo mas relevante para el objetivo es "review" por lo cual este campo tambien se presindé.

In [43]:
# Se crea una variable que contiene los valores unicos del campo 'recommend'
valores_recommend = df_reviews['recommend'].unique() 
valores_recommend

array([True, False], dtype=object)

Con el proceso anterior se cerciora que el campo "recommend" contiene información relevante en todas las filas del DataFrame

#### Fechas

In [55]:
# Se elimina la cadena 'Posted' de todos las filas del campo "posted"
df_reviews['posted'] = df_reviews['posted'].str.replace('Posted', '')
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [56]:
# Se crea un nuevo campo llamado "Fecha" donde se almacenara el contenido de "posted" convertido a formato datetime
df_reviews['Date'] = pd.to_datetime(df_reviews['posted'], dayfirst=True, errors='coerce') # Con estos atributos se logra primero que se reconozca adecuadamente el formato de la fecha en el campo "posted" y segundo que lo que no se reconozca como datetime se convierta en una NaT "Not a Time"
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013-09-08


In [61]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Date" del DataFrame
cant_Not_Date = df_reviews['Date'].isna().sum()
cant_Not_Date

10119

In [62]:
# Se crea un nuevo campor llamado "Year" donde se almacenara solamente el año de la review
df_reviews['Year'] = pd.to_datetime(df_reviews['Date']).dt.year
df_reviews.head(2)

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date,Year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,2011.0


Debido a que se cuenta con valores NaT en el campo "Date" el output del campo "Year" esta en formato float por lo cual se van a tranformar a formato "int"

In [68]:
# Se cuenta la cantidad de frecuencia de valores unicos que posee el campo "Year"
fecuencia_valores_Year = df_reviews['Year'].value_counts()
fecuencia_valores_Year

Year
2014.0    22111
2015.0    18470
2013.0     6792
2012.0     1215
2011.0      532
2010.0       66
Name: count, dtype: int64

In [69]:
# Se crea una variable para contabilizar la cantidad NaT's que hay en el nuevo campo "Year" del DataFrame
cant_Not_Year = df_reviews['Year'].isna().sum()
cant_Not_Year

10119

Con las tranformaciones anteriores se puede establecer que la cantidad de filas donde faltan unicamente el año en el campo "posted" son 10119 y es por esta razón que hay nulos al momento de transformalos a formatos de fecha

In [63]:
df_reviews

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Date,Year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2011.0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,2011.0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,2011.0
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,2014.0
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013-09-08,2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaT,NaN
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaT,NaN
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaT,NaN
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,July 20.,,730,No ratings yet,True,:D,NaT,NaN
